<a href="https://colab.research.google.com/github/Abivankenobi/Slanger/blob/main/Slanger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import tensorflow as tf


df = pd.read_excel("/content/cleaned_words.xlsx")

In [ ]:
df.value_counts("label")

,count
label,
Humor/Sarcasm,421
Chill/Neutral,361
Frustration,252
Positive,198
Negative,179


In [ ]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["clean"])

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["clean"])

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df["clean"]), maxlen=100)
X_val   = pad_sequences(tokenizer.texts_to_sequences(val_df["clean"]), maxlen=100)
X_test  = pad_sequences(tokenizer.texts_to_sequences(test_df["clean"]), maxlen=100)

In [ ]:
y_train, y_val, y_test = train_df["label"], val_df["label"], test_df["label"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode string labels to integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val   = le.transform(y_val)
y_test  = le.transform(y_test)


In [ ]:
num_classes = len(le.classes_)

model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(64, return_sequences=False),
    Dense(64, activation="relu"),
    Dense(num_classes, activation="softmax")
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.3074 - loss: 1.5909 - val_accuracy: 0.3759 - val_loss: 1.5608
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.3792 - loss: 1.5100 - val_accuracy: 0.3759 - val_loss: 1.4902
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.4166 - loss: 1.4089 - val_accuracy: 0.3901 - val_loss: 1.4652
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.5300 - loss: 1.0980 - val_accuracy: 0.3759 - val_loss: 1.8910
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.7107 - loss: 0.8453 - val_accuracy: 0.3759 - val_loss: 1.7668


In [ ]:
import numpy as np

y_pred = np.argmax(model.predict(X_test), axis=1)
predicted_labels = le.inverse_transform(y_pred)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict_sentence(sentence):
    # Convert to sequence
    seq = tokenizer.texts_to_sequences([sentence])
    # Pad to same length as training
    padded = pad_sequences(seq, maxlen=100, padding="post", truncating="post")

    # Get prediction probabilities
    pred = model.predict(padded)
    # Get class index
    pred_class = np.argmax(pred, axis=1)[0]
    # Convert back to string label
    return le.inverse_transform([pred_class])[0]


In [ ]:
print(predict_sentence("ILY"))
print(predict_sentence("fuck, ILY"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Chill/Neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Chill/Neutral
